In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import pandas as pd
from pyspark.sql.types import IntegerType, LongType, DoubleType, BooleanType, DateType

In [2]:
# Probably need to change this if your on MAC
spark = (
    SparkSession.builder.appName("Preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/01 15:30:28 WARN Utils: Your hostname, Andy-PC resolves to a loopback address: 127.0.1.1; using 172.27.185.53 instead (on interface eth0)
24/09/01 15:30:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/01 15:30:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transactions1 = spark.read.parquet("../data/tables/project-2-bnpl-tables-part2/tables/transactions_20210228_20210827_snapshot")
transactions2 = spark.read.parquet("../data/tables/project-2-bnpl-tables-part3/tables/transactions_20210828_20220227_snapshot")
transactions3 = spark.read.parquet("../data/tables/project-2-bnpl-tables-part4/tables/transactions_20220228_20220828_snapshot")
transactions = transactions1.union(transactions2).union(transactions3)
transactions.show()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
|  18479| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|
|      3| 76819856970|  448.529684285612|5ace6a24-cdf0-4aa...|    2021-08-20|
|  18479| 67609108741|  86.4040605836911|d0e180f0-cb06-42a...|    2021-08-20|
|      3| 34096466752| 301.5793450525113|6fb1ff48-24bb-4f9...|    2021-08-20|
|  18482| 70501974849| 68.75486276223054|8505fb33-b69a-412...|    2021-08-20|
|      4| 49891706470| 48.89796461900801|ed11e477-b09f-4ae...|  

In [4]:
tbl_consumer = spark.read.option("header", True).option("delimiter", "|").csv("../data/tables/project-2-bnpl-tables-part1/tables/tbl_consumer.csv")
print("Number of counts before preprocessing: " + str(tbl_consumer.count()))

tbl_consumer = tbl_consumer.drop("address") # dropped address column
tbl_consumer = tbl_consumer.filter(F.col("state").isin(["NSW", "WA", "NT", "VIC", "TAS", "QLD", "SA",   "ACT"]))   
tbl_consumer = tbl_consumer.filter(F.col("gender").isin(["Female", "Male", "Undisclosed"]))
tbl_consumer = tbl_consumer.withColumn("postcode", tbl_consumer["postcode"].cast(IntegerType()))
tbl_consumer = tbl_consumer.withColumn("consumer_id", tbl_consumer["consumer_id"].cast(LongType()))
# Looked up all the valid postcodes in Australia. But some of them are considered LVR or PO boxes only, but I kept them in anyways idk.
tbl_consumer = tbl_consumer.filter(
    ((F.col("postcode") >= 1000) & (F.col("postcode") <= 1999)) |  
    ((F.col("postcode") >= 2000) & (F.col("postcode") <= 2599)) |
    ((F.col("postcode") >= 2619) & (F.col("postcode") <= 2899)) |
    ((F.col("postcode") >= 2921) & (F.col("postcode") <= 2999)) |
    ((F.col("postcode") >= 200) & (F.col("postcode") <= 299)) |  
    ((F.col("postcode") >= 2600) & (F.col("postcode") <= 2618)) |
    ((F.col("postcode") >= 2900) & (F.col("postcode") <= 2920)) |
    ((F.col("postcode") >= 3000) & (F.col("postcode") <= 3999)) |
    ((F.col("postcode") >= 8000) & (F.col("postcode") <= 8999)) |  
    ((F.col("postcode") >= 4000) & (F.col("postcode") <= 4999)) |
    ((F.col("postcode") >= 9000) & (F.col("postcode") <= 9999)) |  
    ((F.col("postcode") >= 5000) & (F.col("postcode") <= 5799)) |
    ((F.col("postcode") >= 5800) & (F.col("postcode") <= 5999)) |  
    ((F.col("postcode") >= 6000) & (F.col("postcode") <= 6797)) |
    ((F.col("postcode") >= 6800) & (F.col("postcode") <= 6999)) |  
    ((F.col("postcode") >= 7000) & (F.col("postcode") <= 7799)) |
    ((F.col("postcode") >= 7800) & (F.col("postcode") <= 7999)) |  
    ((F.col("postcode") >= 800) & (F.col("postcode") <= 899)) |
    ((F.col("postcode") >= 900) & (F.col("postcode") <= 999))
)

print(tbl_consumer.select("state").distinct())
print(tbl_consumer.select("gender").distinct())


print("Number of unique consumer_id: "+str(tbl_consumer.select("consumer_id").distinct().count()))
print("Number of counts after preprocessing: " + str(tbl_consumer.count()))

tbl_consumer.printSchema()
tbl_consumer

Number of counts before preprocessing: 499999
+-----+
|state|
+-----+
|   NT|
|  ACT|
|   SA|
|  TAS|
|   WA|
|  QLD|
|  VIC|
|  NSW|
+-----+

+-----------+
|     gender|
+-----------+
|Undisclosed|
|     Female|
|       Male|
+-----------+

Number of unique consumer_id: 499690
Number of counts after preprocessing: 499690
root
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: long (nullable = true)



name,state,postcode,gender,consumer_id
Yolanda Williams,WA,6935,Female,1195503
Mary Smith,NSW,2782,Female,179208
Jill Jones MD,NT,862,Female,1194530
Lindsay Jimenez,NSW,2780,Female,154128
Rebecca Blanchard,WA,6355,Female,712975
Karen Chapman,NSW,2033,Female,407340
Andrea Jones,QLD,4606,Female,511685
Stephen Williams,WA,6056,Male,448088
Stephanie Reyes,NSW,2482,Female,650435
Jillian Gonzales,VIC,3220,Female,1058499


In [11]:
merchant_fraud = spark.read.option("header", True).csv("../data/tables/project-2-bnpl-tables-part1/tables/merchant_fraud_probability.csv")
print(merchant_fraud.count())

merchant_fraud = merchant_fraud.filter((F.col("fraud_probability")>=0) &(F.col("fraud_probability")<=100))
# Type casting
merchant_fraud = merchant_fraud.withColumn("fraud_probability", merchant_fraud["fraud_probability"].cast(DoubleType()))
merchant_fraud = merchant_fraud.withColumn("merchant_abn", merchant_fraud["merchant_abn"].cast(LongType()))


merchant_fraud = merchant_fraud.withColumn("order_datetime", F.date_format(merchant_fraud["order_datetime"], "yyyy-MM"))    # I dropped the date here cause we said we only need month.
merchant_fraud.printSchema()

merchant_fraud

114
root
 |-- merchant_abn: long (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: double (nullable = true)



merchant_abn,order_datetime,fraud_probability
19492220327,2021-11,44.403658647495355
31334588839,2021-10,42.75530083865367
19492220327,2021-12,38.867790051131095
82999039227,2021-12,94.1347004808891
90918180829,2021-09,43.32551731714902
31334588839,2021-12,38.36165958070444
23686790459,2021-12,79.4543441508535
14827550074,2021-11,46.45775596795885
31334588839,2021-11,36.20971272078342
19492220327,2021-12,33.819672154331755


For here, I looked up what is considered a legit ABN number, and this is code that will execute it, but when its done you can see that there is only 5 entries left so I'm not sure if we should leave it in or not.

In [6]:
def valid_abn(abn_number):
    abn_number = str(abn_number)
    
    if len(abn_number) != 11 or not abn_number.isdigit():
        return False 
    
    digits = [int(i) for i in abn_number]
    
    digits[0] -= 1
    
    weights = [10, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
    
    total = sum(d * w for d, w in zip(digits, weights))
    
    return total % 89 == 0

validate_abn_udf = F.udf(valid_abn, BooleanType())

In [7]:
test = merchant_fraud.withColumn("valid_abn", validate_abn_udf(merchant_fraud["merchant_abn"]))
test = test.filter(F.col("valid_abn") == True)
test.show()

+------------+--------------+------------------+---------+
|merchant_abn|order_datetime| fraud_probability|valid_abn|
+------------+--------------+------------------+---------+
| 11149063370|       2021-08| 56.43761254995139|     true|
| 11149063370|       2021-11|52.407803322764764|     true|
| 11149063370|       2022-02| 51.01538421455241|     true|
| 80518954462|       2021-11|30.024291830924508|     true|
| 30275989942|       2022-01|63.108592876362856|     true|
+------------+--------------+------------------+---------+

